# Integrate Real-Time User Feedback

In this example notebook, we showcase how to integrate real-time feedback for your RAG system using RAG Workbench.

**🌟 Please run this notebook in Google Colab! It generates an interactive thumbs up/down UI that renders better in Colab.🌟**



## Notebook Outline
* [Introduction](#intro)
* [Setup](#setup)
* [Part 1: Log User Feedback](#part1)
  * [Trace your code](#trace_code)
  * [Annotate trace with user feedback](#annotate)
* [Part 2: Analyze User Feedback](#part2)
  * [View user feedback results](#view)
* [More Resources](#more)



<a name="intro"></a>
# Introduction
**User Feedback** is the feedback responses obtained from end users. These responses are typically related to the quality of the output of your application.

RAG Workbench allows you to incorporate user feedback into your trace via the LastMile Tracing SDK, providing valuable insights into how users perceive the performance your RAG system.

User feedback can be captured in various forms:

* **Feedback Score:** Users can rate a specific span, such as a LLM-generated output or a retrieved document (e.g., 👍 / 👎 or on a scale 1-10)
* **Expected Value:** Users can provide the expected or correct output for a span, which is stored in the expected field alongside input and output. This is useful for capturing corrections.
* **User Comment:** Users can share additional context or qualitative feedback using a free-form text field.

<a name="setup"></a>
# Setup

To begin, we need to install lastmile-eval library. We are using ipywidgets for this example so installing that here too.

In [ ]:
!pip install lastmile-eval --upgrade
!pip install ipywidgets

Import all modules used in this tutorial.

In [2]:
from lastmile_eval.rag.debugger.tracing import get_lastmile_tracer
from lastmile_eval.rag.debugger.api import LastMileTracer
from lastmile_eval.rag.debugger.tracing.sdk import get_span_id, get_trace_id
from openai import OpenAI
from IPython.display import display, clear_output
from ipywidgets import Button, HBox, Output

You need to have the following API keys:

1. **LastMile AI API Token:** Get from the [Settings page](https://lastmileai.dev/settings?page=tokens) after creating a free LastMile AI account.
2. **OpenAI API Key:** Create from the [API Keys page](https://platform.openai.com/account/api-keys).

Setup your keys using one of the following methods:

* **Google Colab:** Add secrets `OPENAI_API_KEY` and `LASTMILE_API_TOKEN` in "Secrets Manager" (lock icon on left).
* **Local Notebook:** Create a `.env` file and add a line for each key (ex. `LASTMILE_API_TOKEN=your-api-token`). 

Run the code cell below after setting the keys. Avoid inputting keys directly in the notebook.

In [3]:
import os

try:
    # If running on Google Colab, use userdata to securely input keys
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    LASTMILE_API_TOKEN = userdata.get('LASTMILE_API_TOKEN')
except ModuleNotFoundError:
    # If running locally, load keys from .env file
    from dotenv import load_dotenv
    load_dotenv()
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    LASTMILE_API_TOKEN = os.getenv('LASTMILE_API_TOKEN')

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['LASTMILE_API_TOKEN'] = LASTMILE_API_TOKEN

<a name="part1"></a>

# Part 1: Log User Feedback
In this example, we will generate a riddle with OpenAI's `gpt-3.5-turbo`, and collect user feedback (thumbs up/down) on the riddle.

<a name="trace_code"></a>

## Trace your code
First, instantiate a LastMile Tracer object.

In [5]:
tracer: LastMileTracer = get_lastmile_tracer(
    tracer_name="tracer_0",
    project_name="User-Feedback",
)

The function generates our riddle using `gpt-3.5-turbo`. We've added tracing for this function.

In [6]:
# Decorate function with tracer for monitoring
@tracer.trace_function("llm_call")
def generate_riddle() -> str:
    """
    Generate a riddle using OpenAI's GPT-3.5-turbo model.
    """

    query = "Tell me a riddle and give me the answer."
    prompt = "You must respond in capital letters."
    model = "gpt-3.5-turbo"

    response = OpenAI().chat.completions.create(
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": query}
        ],
        model=model
    )

    riddle = response.choices[0].message.content

    # Store trace and span IDs for linking user feedback to a specific trace/span
    tracer.store_trace_id(key=riddle, span_id=get_span_id(), trace_id=get_trace_id())

    # Log query event to trace
    tracer.add_query_event(
        query=query,
        llm_output=riddle,
        system_prompt=prompt,
        metadata={"model": model}
    )

    return riddle


riddle = generate_riddle()

<a name="annotate"></a>

## Annotate trace with user feedback
We generate a UI element to give a thumbs up/down on our traced llm call and log the reaction to the trace and span.


In [ ]:
import trace


output = Output()

# Retrieve trace_id and span_id to link user feedback to the generated riddle
trace_id, span_id = tracer.read_trace_id(key=riddle)

def create_feedback_button(feedback):
    """Create a button for user feedback."""
    button_style = "success" if feedback == "👍" else "danger"
    button = Button(description=feedback, button_style=button_style)
    button.on_click(lambda _: log_feedback(feedback))
    return button

def log_feedback(feedback):
    """Log user feedback and update the output display."""
    with output:
        clear_output()
        print(f"You selected: {feedback}")
        tracer.log_binary_feedback(value=feedback == "👍", trace_id=trace_id)

def display_feedback_ui():
    """Display the thumbs up and thumbs down buttons for user feedback."""
    thumbsup_button, thumbsdown_button = create_feedback_button("👍"), create_feedback_button("👎")
    display(HBox([thumbsup_button, thumbsdown_button]), output)

# Display the generated riddle and prompt for user feedback
print(f'\nGenerated Riddle:\n{riddle}\nGive it a thumbs up or thumbs down to provide feedback.\n')
display_feedback_ui()

Now our user feedback has been logged to our Trace!

<a name="view"></a>
## View user feedback results
User feedback is 1:1 with a specific trace. You can view user feedback in the 'Traces' page of RAG Workbench.

#### From your terminal:

Export your LASTMILE_API_TOKEN

```bash
export LASTMILE_API_TOKEN="<your-api-token>"
```

Run this CLI command to access the UI

```bash
rag-debug launch
```
Go to the 'Traces' Page where you see all the Traces listed. One of the columns here is 'User Feedback'.
<img width="973" alt="Screenshot 2024-06-13 at 3 26 36 PM" src="https://github.com/lastmile-ai/aiconfig/assets/81494782/6881194d-7fa3-42b2-9764-dfe4dafe2361">

Click on the Trace to get to the Trace Overview Page. You can then click on `llm-call' to access the prompt playground to iterate on the prompt.

<img width="973" alt="Screenshot 2024-06-13 at 3 32 01 PM" src="https://github.com/lastmile-ai/aiconfig/assets/81494782/8e51ab6a-82f2-4a1f-8557-dee44124e68c">

Debug your Prompt

<img width="973" alt="Screenshot 2024-06-13 at 4 26 11 PM" src="https://github.com/lastmile-ai/aiconfig/assets/81494782/cbd61565-b61a-4952-9a07-206cc43e7503">


<a name="more"></a>

# More Resources
* [RAG Workbench Documentation](https://rag.lastmileai.dev/docs/basics)
* [Discord - ask us anything!](https://discord.com/invite/xBhNKTetGx)